In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np

In [2]:
analyser = SentimentIntensityAnalyzer()

In [3]:
news_dir = 'data/bucket/news/'
twitter_dir = 'data/bucket/twitter/'
influencers = ['BarackObama','cnnbrk', 'KimKardashian', 'Trump']
sources = ['New York Times', 'CNN', 'Washington Post']

In [4]:
def sentiment_analyzer_scores(sentence):
    try:
        score = analyser.polarity_scores(sentence)
        return score['compound']
    except:
        print('Failed to score content:',sentence)
        return 0
    
#     print("{:-<40} {}".format(sentence, str(score)))

In [5]:
#TODO ADD CUSTOM HEURISTICS
def heuristic_1(df):
    senti = sum(df['sentiment_score']*df['number_of_likes'].apply(np.log))
    return senti
    

In [6]:
def sentimentTwiiter(influencers):
    for influencer in influencers:
        print('Processing:', influencer)
        df = pd.read_csv(twitter_dir+influencer+'.csv')
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Date'] = pd.to_datetime(df['Date'].dt.normalize())
        df['sentiment_score'] = df['content'].apply(sentiment_analyzer_scores)
        df['number_of_likes'] = df['number_of_likes'].map(lambda x: x if isinstance(x, int) else 0)
        df2 = df.groupby('Date').sum()
        df2 = df2.astype({'number_of_likes': 'int64'})
        df2['heuristic_score'] = df.groupby('Date').apply(heuristic_1)
        df2 = df2.reset_index()
        df2 = df2[['Date','sentiment_score', 'heuristic_score']]
        df2.to_csv(twitter_dir+influencer+'_sentiment.csv', index=False)
    print('done')

In [7]:
def sentimentNews(sources):
    for source in sources:
        print('Processing:', source)
        df = pd.read_csv(news_dir+source+'.csv')
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Date'] = pd.to_datetime(df['Date'].dt.normalize())
        df['sentiment_score'] = df['title'].apply(sentiment_analyzer_scores)
        df2 = df.groupby('Date').sum()
        df2 = df2.reset_index()
        df2 = df2[['Date','sentiment_score']]
        df2.to_csv(news_dir+source+'_sentiment.csv', index=False)
    print('done')

In [8]:
#AUTOMATION
sentimentTwiiter(influencers)
sentimentNews(sources)

Processing: BarackObama
Processing: cnnbrk
Processing: KimKardashian
Processing: Trump
done
Processing: New York Times
Processing: CNN
Processing: Washington Post
done
